# Use VeinsGym with the Straight Scenario

This notebook registers and runs the `straight_scenario` with VeinsGym. The agent observes features relevant for task offloading and returns an action: 0 = no offload, 1/2/3 = offload to RSU 0/1/2.

## Dependencies

You'll need SUMO, OMNeT++, a C++ compiler, ZeroMQ, and the `veins_gym` Python package.

In [1]:
import numpy as np
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_

In [2]:
import gym
import veins_gym

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [3]:
# Ensure OMNeT++ environment is available inside this kernel
import os, subprocess, shutil, sys, pathlib

def source_bash_env(script_path: str):
    cmd = f"bash -lc 'source {script_path} && env -0'"
    out = subprocess.check_output(cmd, shell=True)
    for chunk in out.split(b'\x00'):
        if not chunk: continue
        k, _, v = chunk.partition(b'=')
        if k: os.environ[k.decode()] = v.decode()

omnetpp_setenv = "/home/abhay/omnetpp-5.7.1/setenv"
if os.path.exists(omnetpp_setenv):
    source_bash_env(omnetpp_setenv)
    print('Sourced:', omnetpp_setenv)
else:
    raise FileNotFoundError(f'OMNeT++ setenv not found at {omnetpp_setenv}. Update the path if installed elsewhere.')

print('opp_run path:', shutil.which('opp_run'))
try:
    ver = subprocess.run(['opp_run', '-v'], check=False, capture_output=True, text=True).stdout.splitlines()[0]
    print(ver)
except Exception as e:
    print('Warning: opp_run not callable:', e)

# Preflight checks for straight scenario (now using ../scenario)
base = pathlib.Path.cwd()
sc_dir = (base / '..' / 'scenario').resolve()
print('scenario dir:', sc_dir)
print('omnetpp.ini exists:', (sc_dir / 'omnetpp.ini').exists())
print('run script exists:', (sc_dir / 'run').exists())
print('sumo path:', shutil.which('sumo'))

Sourced: /home/abhay/omnetpp-5.7.1/setenv
opp_run path: /home/abhay/omnetpp-5.7.1/bin/opp_run
OMNeT++ Discrete Event Simulation  (C) 1992-2021 Andras Varga, OpenSim Ltd.
scenario dir: /home/abhay/Major_Project/vanet_task_offloading/scenario
omnetpp.ini exists: True
run script exists: True
sumo path: /usr/bin/sumo


In [4]:
# Register the Straight scenario as a Gym environment (no extra plugin lib needed)
import os, pathlib, subprocess, shutil
base = pathlib.Path.cwd()
sc_dir = (base / '..' / 'scenario').resolve()
assert (sc_dir / 'omnetpp.ini').exists(), 'Missing scenario/omnetpp.ini'
gym.register(
    id='veins-straight-v1',
    entry_point='veins_gym:VeinsEnv',
    kwargs={
        'scenario_dir': '../scenario',
        'timeout': 7.0,
        'print_veins_stdout': True,
        # 'user_interface': 'Cmdenv',
        'config': 'StraightRoad',
        # "run_veins": False,  # do not start veins through Veins-Gym
		# "port": 5555, 
    },
)
print('Registered veins-straight-v1 with scenario_dir:', sc_dir)

Registered veins-straight-v1 with scenario_dir: /home/abhay/Major_Project/vanet_task_offloading/scenario


Create the environment and step it with random actions for a quick smoke test.

In [5]:
env = gym.make('veins-straight-v1')

## Random Agent

In [14]:
obs = env.reset()

# Define observation headers matching the new 11-D vector from C++
headers = [
    "speed(m/s)", "d0(m)", "d1(m)", "d2(m)", "taskSize(MB)",
    "rsu0Busy", "rsu1Busy", "rsu2Busy", "ul0(Mbps)", "ul1(Mbps)", "ul2(Mbps)"
]

def print_obs(step, obs_vec):
    vals = [float(x) for x in np.asarray(obs_vec).tolist()]
    # Header row
    print(f"Step:{step}")
    print("  " + " ".join(f"{h:>11}" for h in headers))
    # Values row (2 decimals)
    print("  " + " ".join(f"{v:>11.2f}" for v in vals))
    print()

# Print initial observation neatly
print_obs(0, obs)

# Rollout
done = False
rewards = []
steps = 0
while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    # Print reward when non-zero
    if reward != 0.0:
        print(f"Received reward: {round(float(reward), 2)}")
        rewards.append(reward)

    # Print observation table when a task is present (taskSize > 0)
    if float(obs[4]) >= 10.0:
        print_obs(steps, obs)
    steps += 1

print('Number of steps:', steps)
print('Mean reward:', float(np.mean(rewards)) if rewards else 0.0)

Step:0
   speed(m/s)       d0(m)       d1(m)       d2(m) taskSize(MB)    rsu0Busy    rsu1Busy    rsu2Busy   ul0(Mbps)   ul1(Mbps)   ul2(Mbps)
        16.67      276.30      775.46     1275.28       15.68        0.00        0.00        0.00       74.19       44.98       31.68

Received reward: 0.13
Step:76
   speed(m/s)       d0(m)       d1(m)       d2(m) taskSize(MB)    rsu0Busy    rsu1Busy    rsu2Busy   ul0(Mbps)   ul1(Mbps)   ul2(Mbps)
        16.67      160.56      658.85     1158.62       16.33        0.00        0.00        0.00       89.80       49.50       34.16

Received reward: 0.16
Step:139
   speed(m/s)       d0(m)       d1(m)       d2(m) taskSize(MB)    rsu0Busy    rsu1Busy    rsu2Busy   ul0(Mbps)   ul1(Mbps)   ul2(Mbps)
        16.67       49.49      542.28     1041.96       14.18        0.00        0.00        0.00      123.73       54.97       36.96

Received reward: 0.27
Step:177
   speed(m/s)       d0(m)       d1(m)       d2(m) taskSize(MB)    rsu0Busy    rsu1Busy    r

## Local processing agent

In [13]:
obs = env.reset()

# Define observation headers matching the new 11-D vector from C++
headers = [
    "speed(m/s)", "d0(m)", "d1(m)", "d2(m)", "taskSize(MB)",
    "rsu0Busy", "rsu1Busy", "rsu2Busy", "ul0(Mbps)", "ul1(Mbps)", "ul2(Mbps)"
]

def print_obs(step, obs_vec):
    vals = [float(x) for x in np.asarray(obs_vec).tolist()]
    # Header row
    print(f"Step:{step}")
    print("  " + " ".join(f"{h:>11}" for h in headers))
    # Values row (2 decimals)
    print("  " + " ".join(f"{v:>11.2f}" for v in vals))
    print()

# Print initial observation neatly
print_obs(0, obs)

# Rollout
done = False
rewards = []
steps = 0
while not done:
    action = 0
    obs, reward, done, info = env.step(action)
    # Print reward when non-zero
    if reward != 0.0:
        print(f"Received reward: {round(float(reward), 2)}")
        rewards.append(reward)

    # Print observation table when a task is present (taskSize > 0)
    if float(obs[4]) >= 10.0:
        print_obs(steps, obs)
    steps += 1

print('Number of steps:', steps)
print('Mean reward:', float(np.mean(rewards)) if rewards else 0.0)

Step:0
   speed(m/s)       d0(m)       d1(m)       d2(m) taskSize(MB)    rsu0Busy    rsu1Busy    rsu2Busy   ul0(Mbps)   ul1(Mbps)   ul2(Mbps)
        16.67      276.30      775.46     1275.28       15.68        0.00        0.00        0.00       74.19       44.98       31.68

Received reward: 0.02
Step:496
   speed(m/s)       d0(m)       d1(m)       d2(m) taskSize(MB)    rsu0Busy    rsu1Busy    rsu2Busy   ul0(Mbps)   ul1(Mbps)   ul2(Mbps)
        16.67      542.49       49.66      458.95       17.85        0.00        0.00        0.00       54.96      123.63       59.69

Number of steps: 898
Mean reward: 0.020139062777161598


## Greedy RSU Agent

In [ ]:
from nearest_free_rsu_agent import pick_action

obs = env.reset()

headers = [
    "speed(m/s)", "d0(m)", "d1(m)", "d2(m)", "taskSize(MB)",
    "rsu0Busy", "rsu1Busy", "rsu2Busy", "ul0(Mbps)", "ul1(Mbps)", "ul2(Mbps)"
]

def print_obs(step, obs_vec):
    vals = [float(x) for x in np.asarray(obs_vec).tolist()]

    print(f"Step:{step}")
    print("  " + " ".join(f"{h:>11}" for h in headers))
    print("  " + " ".join(f"{v:>11.2f}" for v in vals))
    print()

# Print initial observation neatly
print_obs(0, obs)

# Rollout
done = False
rewards = []
steps = 0
while not done:
    action = pick_action(obs)
    obs, reward, done, info = env.step(action)
    # Print reward when non-zero
    if reward != 0.0:
        print(f"Received reward: {round(float(reward), 2)}")
        rewards.append(reward)

    # Print observation table when a task is present (taskSize > 0)
    if float(obs[4]) >= 10.0:
        print_obs(steps, obs)
    steps += 1

print('Number of steps:', steps)
print('Mean reward:', float(np.mean(rewards)) if rewards else 0.0)

Step:0
   speed(m/s)       d0(m)       d1(m)       d2(m) taskSize(MB)    rsu0Busy    rsu1Busy    rsu2Busy   ul0(Mbps)   ul1(Mbps)   ul2(Mbps)
        16.67      276.30      775.46     1275.28       15.68        0.00        0.00        0.00       74.19       44.98       31.68

Received reward: 0.2
Step:49
   speed(m/s)       d0(m)       d1(m)       d2(m) taskSize(MB)    rsu0Busy    rsu1Busy    rsu2Busy   ul0(Mbps)   ul1(Mbps)   ul2(Mbps)
        16.67      193.51      692.17     1191.95       11.78        0.00        0.00        0.00       84.42       48.13       33.42

Received reward: 0.28
Step:85
   speed(m/s)       d0(m)       d1(m)       d2(m) taskSize(MB)    rsu0Busy    rsu1Busy    rsu2Busy   ul0(Mbps)   ul1(Mbps)   ul2(Mbps)
        16.67      144.15      642.20     1141.95       10.81        0.00        0.00        0.00       92.90       50.22       34.54

Received reward: 0.32
Step:117
   speed(m/s)       d0(m)       d1(m)       d2(m) taskSize(MB)    rsu0Busy    rsu1Busy    rsu